In [1]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# TRAINING DATA

In [2]:
# check data dimensions of a selected file
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTr"
os.chdir(file_path)

file_name = "S02.nii"
label = nib.load(file_name)
data_label = label.get_fdata()

print("Dimensions of label data:")
print(data_label.shape)

Dimensions of label data:
(512, 512, 265)


Each file has diffent number of scans and therefore different z-dimension. Need to normalise the data: (only have to do this once, hence why it's commented out)

In [3]:
from scipy import ndimage

file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTr"
os.chdir(file_path)
training_labels_file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\Endpoint locations.xlsx"
training_labels = pd.read_excel(training_labels_file_path, sheet_name="labelsTr", header=None, names = ["labelsTr", "x_1", "y_1", "z_1", "x_2", "y_2", "z_2", "x_3", "y_3", "z_3"])
training_labels = pd.DataFrame(training_labels)
training_labels = training_labels.drop([0,1])
training_labels = training_labels.set_index("labelsTr")
training_labels
labelsTr = list(training_labels.index.values)

for item in labelsTr:

    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    affine = label.affine

    #Resize across z-axis
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current dimensions
    current_depth = data_label.shape[-1]
    current_width = data_label.shape[0]
    current_height = data_label.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    data_label = ndimage.zoom(data_label, (width_factor, height_factor, depth_factor), order=1)
    
    new_label = nib.Nifti1Image(data_label, affine)
    nib.save(new_label, file_name)

In [4]:
training_labels

,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3
labelsTr,,,,,,,,,
S02,276,195,130,202,264,154,340,314,143
S03,345,205,170,227,264,172,335,314,205
S04,289,205,185,184,272,172,304,294,198
S08,354,193,80,170,263,97,338,351,95
S09,292,263,233,189,319,257,348,356,198
...,...,...,...,...,...,...,...,...,...
S120,280,178,582,206,263,638,335,288,602
S121,291,183,306,202,247,286,301,296,293
S122,308,126,178,188,266,217,363,325,218


Check dimensions of a sample file

In [5]:
file_name = "S02.nii"
label = nib.load(file_name)
data_label = label.get_fdata()

print("Dimensions of label data:")
print(data_label.shape)

Dimensions of label data:
(128, 128, 64)


have normalised image data. Have to do this for label data too. So for example if z index for an endpoint is "z_old" then z_new = desired_depth/current_depth = desired_depth/z_old

In [6]:
# Choose training, validation or testing labels folder
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL UN-NORMALISED DATA\labelsTr" # need this file path of original un-normalised data to get the old dimensions such that the labels can be scaled correctly now that the data in folder "labelled data" has had its dimsnsions changed.
os.chdir(file_path)

labels = list(training_labels.index.values)
labels_normalised = training_labels.copy()

index = 0

for item in labels:
    
    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    shape = data_label.shape
    
    current_depth = data_label.shape[-1]
    current_width = data_label.shape[0]
    current_height = data_label.shape[1] 
    
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    
    labels_normalised["x_1"][index] = labels_normalised["x_1"][index]*(desired_width/current_width)
    labels_normalised["x_2"][index] = labels_normalised["x_2"][index]*(desired_width/current_width)
    labels_normalised["x_3"][index] = labels_normalised["x_3"][index]*(desired_width/current_width)
    
    labels_normalised["y_1"][index] = labels_normalised["y_1"][index]*(desired_height/current_height)
    labels_normalised["y_2"][index] = labels_normalised["y_2"][index]*(desired_height/current_height)
    labels_normalised["y_3"][index] = labels_normalised["y_3"][index]*(desired_height/current_height)
    
    labels_normalised["z_1"][index] = labels_normalised["z_1"][index]*(desired_depth/current_depth)
    labels_normalised["z_2"][index] = labels_normalised["z_2"][index]*(desired_depth/current_depth)
    labels_normalised["z_3"][index] = labels_normalised["z_3"][index]*(desired_depth/current_depth)
    
    index += 1

file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTr"
os.chdir(file_path)

In [7]:
labels_normalised

,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3
labelsTr,,,,,,,,,
S02,69.0,48.75,31.396226,50.5,66.0,37.192453,85.0,78.5,34.535849
S03,86.25,51.25,33.272171,56.75,66.0,33.663609,83.75,78.5,40.122324
S04,72.25,51.25,37.827476,46.0,68.0,35.169329,76.0,73.5,40.485623
S08,88.5,48.25,23.925234,42.5,65.75,29.009346,84.5,87.75,28.411215
S09,73.0,65.75,34.598608,47.25,79.75,38.162413,87.0,89.0,29.401392
...,...,...,...,...,...,...,...,...,...
S120,70.0,44.5,33.831063,51.5,65.75,37.086285,83.75,72.0,34.993642
S121,72.75,45.75,39.724138,50.5,61.75,37.127789,75.25,74.0,38.036511
S122,77.0,31.5,28.623116,47.0,66.5,34.894472,90.75,81.25,35.055276


In [8]:
labels_normalised = labels_normalised.astype(float).round(0)
labels_normalised = labels_normalised.astype(int)
labels_normalised.to_csv(r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\training_labels_df"+".csv", index=False)

### Create the training/validation dataset

In [9]:
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTr"
os.chdir(file_path)

labels = list(labels_normalised.index.values)

for item in labels:
    
    # image/segment
    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    
    # label
    x1 = labels_normalised.loc[item,"y_1"] # change x and y due to way image is rotated when read into python vs when read into slicer where label data was acquired (difference of 90 deg, therefore x becomes y and vice versa)
    y1 = labels_normalised.loc[item,"x_1"]
    z1 = labels_normalised.loc[item,"z_1"]

    x2 = labels_normalised.loc[item,"y_2"]
    y2 = labels_normalised.loc[item,"x_2"]
    z2 = labels_normalised.loc[item,"z_2"]

    x3 = labels_normalised.loc[item,"y_3"]
    y3 = labels_normalised.loc[item,"x_3"]
    z3 = labels_normalised.loc[item,"z_3"]
    
    # you must find a way to save the image and the coordinate labels together in a file such that they can be called in together for model
    
    # Create a new data frame with endpoint coordinates of one patient's data
    #df = pd.DataFrame(np.array([[x1, y1, z1],[x2, y2, z2],[x3, y3, z3]]), columns=['x', 'y', 'z'], index=[1,2,3]) # index corresponds as: 1=endpoint1, 2=endpoint2, 3=endpoint3.
    df = pd.DataFrame(np.array([[x1, y1, z1, x2, y2, z2, x3, y3, z3]]), columns=['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3']) 
    
    # Assign the data frame to a variable with the name SXX + "_labels"
    df_name = item + "_labels"
    exec("%s = %s" % (df_name, 'df'))

    # Print the name of the variable and the first few rows of the data frame
    #print(df_name)
    #print(eval(df_name))
    
    # Save each data frame produced in same file as normalised nifti data with file name "SXX_labels"
    df.to_csv(file_path+"/"+df_name+".csv", index=False)

#### Note: orginal labelsTr file remains the same as it was orginally. The copy that was put in folder "labelled data" has the new dimensions. You will have to repeat this for the validation & testing data.

In [10]:
#training_labels_file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\Endpoint locations.xlsx"
#training_labels = pd.read_excel(training_labels_file_path, sheet_name="labelsTr", header=None, names = ["labelsTr", "x_1", "y_1", "z_1", "x_2", "y_2", "z_2", "x_3", "y_3", "z_3"])
#training_labels = pd.DataFrame(training_labels)
#training_labels = training_labels.drop([0,1])
#training_labels = training_labels.set_index("labelsTr")
#training_labels
#training_labels.to_csv(r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\training_labels_df_org"+".csv", index=False)

# VALIDATION DATA

In [11]:
# check data dimensions of a selected file
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsVa"
os.chdir(file_path)

file_name = "S01.nii"
label = nib.load(file_name)
data_label = label.get_fdata()

print("Dimensions of label data:")
print(data_label.shape)

Dimensions of label data:
(512, 512, 295)


In [12]:
from scipy import ndimage

file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsVa"
os.chdir(file_path)
validation_labels_file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\Endpoint locations.xlsx"
validation_labels = pd.read_excel(validation_labels_file_path, sheet_name="labelsVa", header=None, names = ["labelsVa", "x_1", "y_1", "z_1", "x_2", "y_2", "z_2", "x_3", "y_3", "z_3"])
validation_labels = pd.DataFrame(validation_labels)
validation_labels = validation_labels.drop([0,1])
validation_labels = validation_labels.set_index("labelsVa")
validation_labels
labelsVa = list(validation_labels.index.values)

for item in labelsVa:

    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    affine = label.affine

    #Resize across z-axis
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current dimensions
    current_depth = data_label.shape[-1]
    current_width = data_label.shape[0]
    current_height = data_label.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    data_label = ndimage.zoom(data_label, (width_factor, height_factor, depth_factor), order=1)
    
    new_label = nib.Nifti1Image(data_label, affine)
    nib.save(new_label, file_name)

In [13]:
file_name = "S01.nii"
label = nib.load(file_name)
data_label = label.get_fdata()

print("Dimensions of label data:")
print(data_label.shape)

Dimensions of label data:
(128, 128, 64)


In [14]:
# Choose training, validation or testing labels folder
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL UN-NORMALISED DATA\labelsVa" # need this file path of original un-normalised data to get the old dimesnsions such that the labels can be scaled correctly now that the data in folder "labelled data" has had its dimsnsions changed.
os.chdir(file_path)

# pick training or validation
labels = list(validation_labels.index.values)

# pick training or validation
labels_normalised = validation_labels.copy()

index = 0

for item in labels:
    
    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    shape = data_label.shape
    
    current_depth = data_label.shape[-1]
    current_width = data_label.shape[0]
    current_height = data_label.shape[1] 
    
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    
    labels_normalised["x_1"][index] = labels_normalised["x_1"][index]*(desired_width/current_width)
    labels_normalised["x_2"][index] = labels_normalised["x_2"][index]*(desired_width/current_width)
    labels_normalised["x_3"][index] = labels_normalised["x_3"][index]*(desired_width/current_width)
    
    labels_normalised["y_1"][index] = labels_normalised["y_1"][index]*(desired_height/current_height)
    labels_normalised["y_2"][index] = labels_normalised["y_2"][index]*(desired_height/current_height)
    labels_normalised["y_3"][index] = labels_normalised["y_3"][index]*(desired_height/current_height)
    
    labels_normalised["z_1"][index] = labels_normalised["z_1"][index]*(desired_depth/current_depth)
    labels_normalised["z_2"][index] = labels_normalised["z_2"][index]*(desired_depth/current_depth)
    labels_normalised["z_3"][index] = labels_normalised["z_3"][index]*(desired_depth/current_depth)
    
    index += 1

# pick training, validation or testing
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsVa"
os.chdir(file_path)

In [15]:
labels_normalised = labels_normalised.astype(float).round(0)
labels_normalised = labels_normalised.astype(int)
labels_normalised.to_csv(r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\validation_labels_df"+".csv", index=False)

In [16]:
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsVa"
os.chdir(file_path)

labels = list(labels_normalised.index.values)

for item in labels:
    
    # image/segment
    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    
    # label
    x1 = labels_normalised.loc[item,"y_1"] # change x and y due to way image is rotated when read into python vs when read into slicer where label data was acquired (difference of 90 deg, therefore x becomes y and vice versa)
    y1 = labels_normalised.loc[item,"x_1"]
    z1 = labels_normalised.loc[item,"z_1"]

    x2 = labels_normalised.loc[item,"y_2"]
    y2 = labels_normalised.loc[item,"x_2"]
    z2 = labels_normalised.loc[item,"z_2"]

    x3 = labels_normalised.loc[item,"y_3"]
    y3 = labels_normalised.loc[item,"x_3"]
    z3 = labels_normalised.loc[item,"z_3"]
    
    # you must find a way to save the image and the coordinate labels together in a file such that they can be called in together for model
    
    # Create a new data frame with endpoint coordinates of one patient's data
    #df = pd.DataFrame(np.array([[x1, y1, z1],[x2, y2, z2],[x3, y3, z3]]), columns=['x', 'y', 'z'], index=[1,2,3]) # index corresponds as: 1=endpoint1, 2=endpoint2, 3=endpoint3.
    df = pd.DataFrame(np.array([[x1, y1, z1, x2, y2, z2, x3, y3, z3]]), columns=['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3']) 
    
    # Assign the data frame to a variable with the name SXX + "_labels"
    df_name = item + "_labels"
    exec("%s = %s" % (df_name, 'df'))

    # Print the name of the variable and the first few rows of the data frame
    #print(df_name)
    #print(eval(df_name))
    
    # Save each data frame produced in same file as normalised nifti data with file name "SXX_labels"
    df.to_csv(file_path+"/"+df_name+".csv", index=False)

# TESTING DATA

In [17]:
# check data dimensions of a selected file
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTe"
os.chdir(file_path)

file_name = "S130.nii"
label = nib.load(file_name)
data_label = label.get_fdata()

print("Dimensions of label data:")
print(data_label.shape)

Dimensions of label data:
(512, 512, 411)


In [18]:
from scipy import ndimage

file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTe"
os.chdir(file_path)
testing_labels_file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\Endpoint locations.xlsx"
testing_labels = pd.read_excel(testing_labels_file_path, sheet_name="labelsTe", header=None, names = ["labelsTe", "x_1", "y_1", "z_1", "x_2", "y_2", "z_2", "x_3", "y_3", "z_3"])
testing_labels = pd.DataFrame(testing_labels)
testing_labels = testing_labels.drop([0,1])
testing_labels = testing_labels.set_index("labelsTe")
testing_labels
labelsTe = list(testing_labels.index.values)

for item in labelsTe:

    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    affine = label.affine

    #Resize across z-axis
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current dimensions
    current_depth = data_label.shape[-1]
    current_width = data_label.shape[0]
    current_height = data_label.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    data_label = ndimage.zoom(data_label, (width_factor, height_factor, depth_factor), order=1)
    
    new_label = nib.Nifti1Image(data_label, affine)
    nib.save(new_label, file_name)

In [19]:
file_name = "S130.nii"
label = nib.load(file_name)
data_label = label.get_fdata()

print("Dimensions of label data:")
print(data_label.shape)

Dimensions of label data:
(128, 128, 64)


In [20]:
# Choose training, validation or testing labels folder
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL UN-NORMALISED DATA\labelsTe" # need this file path of original un-normalised data to get the old dimesnsions such that the labels can be scaled correctly now that the data in folder "labelled data" has had its dimsnsions changed.
os.chdir(file_path)

# pick training or validation
labels = list(testing_labels.index.values)

# pick training or validation
labels_normalised = testing_labels.copy()

index = 0

for item in labels:
    
    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    shape = data_label.shape
    
    current_depth = data_label.shape[-1]
    current_width = data_label.shape[0]
    current_height = data_label.shape[1] 
    
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    
    labels_normalised["x_1"][index] = labels_normalised["x_1"][index]*(desired_width/current_width)
    labels_normalised["x_2"][index] = labels_normalised["x_2"][index]*(desired_width/current_width)
    labels_normalised["x_3"][index] = labels_normalised["x_3"][index]*(desired_width/current_width)
    
    labels_normalised["y_1"][index] = labels_normalised["y_1"][index]*(desired_height/current_height)
    labels_normalised["y_2"][index] = labels_normalised["y_2"][index]*(desired_height/current_height)
    labels_normalised["y_3"][index] = labels_normalised["y_3"][index]*(desired_height/current_height)
    
    labels_normalised["z_1"][index] = labels_normalised["z_1"][index]*(desired_depth/current_depth)
    labels_normalised["z_2"][index] = labels_normalised["z_2"][index]*(desired_depth/current_depth)
    labels_normalised["z_3"][index] = labels_normalised["z_3"][index]*(desired_depth/current_depth)
    
    index += 1

# pick training, validation or testing
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTe"
os.chdir(file_path)

In [21]:
labels_normalised = labels_normalised.astype(float).round(0)
labels_normalised = labels_normalised.astype(int)
labels_normalised.to_csv(r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\testing_labels_df"+".csv", index=False)

In [22]:
file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL DATA SET\labelsTe"
os.chdir(file_path)

labels = list(labels_normalised.index.values)

for item in labels:
    
    # image/segment
    file_name = item + ".nii"
    label = nib.load(file_name)
    data_label = label.get_fdata()
    
    # label
    x1 = labels_normalised.loc[item,"y_1"] # change x and y due to way image is rotated when read into python vs when read into slicer where label data was acquired (difference of 90 deg, therefore x becomes y and vice versa)
    y1 = labels_normalised.loc[item,"x_1"]
    z1 = labels_normalised.loc[item,"z_1"]

    x2 = labels_normalised.loc[item,"y_2"]
    y2 = labels_normalised.loc[item,"x_2"]
    z2 = labels_normalised.loc[item,"z_2"]

    x3 = labels_normalised.loc[item,"y_3"]
    y3 = labels_normalised.loc[item,"x_3"]
    z3 = labels_normalised.loc[item,"z_3"]
    
    # you must find a way to save the image and the coordinate labels together in a file such that they can be called in together for model
    
    # Create a new data frame with endpoint coordinates of one patient's data
    #df = pd.DataFrame(np.array([[x1, y1, z1],[x2, y2, z2],[x3, y3, z3]]), columns=['x', 'y', 'z'], index=[1,2,3]) # index corresponds as: 1=endpoint1, 2=endpoint2, 3=endpoint3.
    df = pd.DataFrame(np.array([[x1, y1, z1, x2, y2, z2, x3, y3, z3]]), columns=['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3']) 
    
    # Assign the data frame to a variable with the name SXX + "_labels"
    df_name = item + "_labels"
    exec("%s = %s" % (df_name, 'df'))

    # Print the name of the variable and the first few rows of the data frame
    #print(df_name)
    #print(eval(df_name))
    
    # Save each data frame produced in same file as normalised nifti data with file name "SXX_labels"
    df.to_csv(file_path+"/"+df_name+".csv", index=False)

In [3]:
#testing_labels_file_path = r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\Endpoint locations.xlsx"
#testing_labels = pd.read_excel(testing_labels_file_path, sheet_name="labelsTe", header=None, names = ["labelsTe", "x_1", "y_1", "z_1", "x_2", "y_2", "z_2", "x_3", "y_3", "z_3"])
#testing_labels = pd.DataFrame(testing_labels)
#testing_labels = testing_labels.drop([0,1])
#testing_labels = testing_labels.set_index("labelsTe")
#testing_labels
#testing_labels.to_csv(r"C:\Users\tombe\OneDrive - University of Bath\Documents\YEAR 4\FYP\Semester 2\Data\FINAL UN-NORMALISED DATA\test_labels_df_un_norm_all_data"+".csv", index=False)